In [1]:
import __init__
from tensorflow.keras.utils import Sequence
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from tensorflow.keras.layers import Concatenate,Embedding ,Dense ,Input,LSTM,Permute,Softmax,Lambda,Flatten,GRU,Dropout,BatchNormalization, Normalization, Attention, Bidirectional, Masking
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
!pip install keras_self_attention
from keras_self_attention import SeqSelfAttention
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [3]:
root_path = '../../..'
class DataGenerator(Sequence):

    def __init__(self, meta_df, batch_size=64, time_series=30, input_shape=(20,13), use_full=False,**kwargs):
        super().__init__(**kwargs)
        self.ds_pointer = 0
        self.batch_size = batch_size
        self.meta_df = meta_df
        self.time_series = time_series
        self.use_full = use_full
        self.data = self.load_data()
        self.input_buffer = np.zeros((batch_size,input_shape[0],input_shape[1]))
        self.label_buffer = np.zeros((batch_size,input_shape[0]))
        self.reset_pointer()

    def load_data(self):
        #self.meta_df['Label'] = self.meta_df['Label'].astype(int)
        path = self.meta_df['Path']
        temp = {}
        for p in tqdm(path):
           temp[p] = pd.read_csv(f'{root_path}/{p}').to_numpy()
        return temp

    def reset_pointer(self, seed = 1314):
        self.meta_df = self.meta_df.sample(frac=1, random_state=seed)
        self.ds_pointer = 0

    def __len__(self):
      if self.use_full:
        return len(self.meta_df)// self.batch_size
      else:
        return len(self.meta_df)// self.batch_size// 50

    def __getitem__(self, index):
        if self.ds_pointer + self.batch_size >= len(self.meta_df):
            self.reset_pointer()
            return self.__getitem__(index)
        to_get = self.meta_df.iloc[self.ds_pointer:self.ds_pointer+self.batch_size]
        to_get_path = to_get['Path']
        to_get_label = to_get['Label'].to_numpy()

        for i, label in enumerate(to_get_label):
          self.input_buffer[i, :, :] = self.data[to_get_path.iloc[i]][:,:-1]
          self.label_buffer[i, :] = self.data[to_get_path.iloc[i]][:,-1]

        batch_x = self.input_buffer
        batch_y = self.label_buffer

        self.ds_pointer+=self.batch_size

        return batch_x, batch_y

In [30]:
from strategy.CustomStrategies.MA_Strategy_adjustment_short import Strategy_MA_adjust_short
from strategy.CustomStrategies.MA_Strategy import Strategy_MA

strategy = Strategy_MA
batch_size = 8
input_shape = (20,13)
meta_path = f'{root_path}/ouput/analysis/{strategy.strategy_name}/meta.csv'
meta_df = pd.read_csv(meta_path)

train_meta, test_meta = train_test_split(meta_df)

In [31]:
train_data = DataGenerator(train_meta, batch_size=batch_size, input_shape=input_shape)
test_data = DataGenerator(test_meta, batch_size=batch_size, input_shape=input_shape, use_full=True)

  0%|          | 0/8249 [00:00<?, ?it/s]

  0%|          | 0/2750 [00:00<?, ?it/s]

In [17]:
train_data.__getitem__(0)[1].shape

(64, 20)

In [39]:

def r2_score(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

def build_model(time_series,num_of_features):
    K.clear_session()
    input_ = Input(shape=(time_series,num_of_features), name='Input')
    input_dense = Dense(256)(input_)
    # Define GRU layer
    encoder = GRU(256, return_state=True, return_sequences=True, name='Encoder')
    encoder_outputs, state_h = encoder(input_dense)

    # # Define attention layer
    attention = SeqSelfAttention()(encoder_outputs)

    # Concatenate context vector and encoder outputs
    concat_layer = Concatenate(axis=-1, name='Concatenate')
    decoder_combined_context = concat_layer([attention, input_dense])

    # Define decoder
    decoder_gru = GRU(256, return_sequences=True, name='Decoder')

    # Pass the concatenated input through the decoder
    decoder_output = decoder_gru(decoder_combined_context, initial_state=state_h)

    output = Flatten(name='Flatten')(decoder_output)
    output = Dropout(0.5)(output)
    output = BatchNormalization()(output)
    output = Dense(256,activation="relu")(output)
    output = Dropout(0.5)(output)
    output = BatchNormalization()(output)
    output = Dense(128, activation='relu')(output)
    output = Dropout(0.5)(output)
    output = BatchNormalization()(output)
    output = Dense(time_series, activation='sigmoid')(output)


    model = Model(inputs = input_ , outputs = output)
    adam_optimizer = Adam(learning_rate=0.001, clipnorm=1., weight_decay=1e-7)
    model.compile(loss="mse",optimizer=adam_optimizer,metrics=[r2_score])
    model.summary()
    return model
model = build_model(input_shape[0],input_shape[1])

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)  │ (None, 20, 13)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 20, 256)   │      3,584 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Encoder (GRU)       │ [(None, 20, 256), │    394,752 │ dense[0][0]       │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ seq_self_attention  │ (None, 20, 256)   │     16,449 │ Encoder[0][0]     │
│ (SeqSelfAttention)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Concatenate         │ (None, 20, 512)   │          0 │ seq_self_attenti… │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder (GRU)       │ (None, 20, 256)   │    591,360 │ Concatenate[0][0… │
│                     │                   │            │ Encoder[0][1]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Flatten (Flatten)   │ (None, 5120)      │          0 │ Decoder[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 5120)      │          0 │ Flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 5120)      │     20,480 │ dropout[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │  1,310,976 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dropout_1[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │     32,896 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dropout_2[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 20)        │      2,580 │ batch_normalizat… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,374,613 (9.06 MB)

 Trainable params: 2,363,605 (9.02 MB)

 Non-trainable params: 11,008 (43.00 KB)

: 

In [36]:
# Directory Config
neural_path = 'neural_network'
log_path = 'neural_network/log'
save_path_dir='attention_t20_encode_decode'

if not os.path.exists(log_path):
    os.makedirs(log_path)
checkpoint = ModelCheckpoint(
    f'{neural_path}/{save_path_dir}/saved_model.keras',
    verbose=1,
    monitor='val_r2_score',
    save_weights_only=False,
    save_best_only=True,
    mode='max')

earlystop = EarlyStopping(
    monitor="val_r2_score",
    min_delta=0,
    patience=20,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=False
)
!mkdir neural_network

tensorboard = TensorBoard(log_dir=log_path)

�l�ؿ����ɮ� neural_network �w�g�s�b�C


In [37]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

AttributeError: module 'pydot' has no attribute 'InvocationException'

In [38]:
history = model.fit(train_data, batch_size=batch_size, epochs=2000, validation_data=test_data, callbacks=[checkpoint, earlystop, tensorboard])
plt.plot(history.history['r2_score'])
plt.plot(history.history['val_r2_score'])
plt.title('model R2')
plt.ylabel('R2')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
print(f"Highest Val R2: {max(history.history['val_r2_score'])}")

Epoch 1/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.5928 - r2_score: -45.4291
Epoch 1: val_r2_score improved from -inf to -22.57481, saving model to neural_network/attention_t20_encode_decode/saved_model.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 150ms/step - loss: 0.5918 - r2_score: -44.6925 - val_loss: 0.7135 - val_r2_score: -22.5748
Epoch 2/2000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.5305 - r2_score: -27.6449
Epoch 2: val_r2_score improved from -22.57481 to -20.94496, saving model to neural_network/attention_t20_encode_decode/saved_model.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - loss: 0.5316 - r2_score: -28.1703 - val_loss: 0.7066 - val_r2_score: -20.9450
Epoch 3/2000
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.5713 - r2_score: -18.3929
Epoch 3: val_r2_score improved from -20.94496 to -20.90492, saving model to neural_network/attention_t20_encode_decode/saved_model.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - loss: 0.5791 - r2_score: -18.5707 - val_lo

KeyboardInterrupt: 